In [5]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

In [3]:
print("Test Python naja EiEi")

Test Python naja EiEi


In [9]:
from companies.models import *

In [11]:
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [13]:
from django.db.models import Count, F, Value
from django.db.models.functions import Length, Upper
from django.db.models.lookups import GreaterThan

In [37]:
value = Company.objects.filter(num_employees__gt=F("num_chairs"))
value = Company.objects.filter(num_employees__gt=F("num_chairs") * 2)
value = Company.objects.filter(num_employees__gt=F("num_chairs") + F("num_tables"))
company = (
     Company.objects.filter(num_employees__gt=F("num_chairs"))
     .annotate(chairs_needed=F("num_employees") - F("num_chairs"))
)
print(company.query)

SELECT "companies_company"."id", "companies_company"."name", "companies_company"."ticker", "companies_company"."num_employees", "companies_company"."num_tables", "companies_company"."num_chairs", ("companies_company"."num_employees" - "companies_company"."num_chairs") AS "chairs_needed" FROM "companies_company" WHERE "companies_company"."num_employees" > ("companies_company"."num_chairs")


In [42]:
company = Company.objects.create(name="Google", ticker=Upper(Value("goog")), num_employees=120, num_chairs=150, num_tables=60)
print(company.ticker)

Upper(Value('goog'))


In [43]:
Company.objects.order_by(Length("name").asc())

<QuerySet [<Company: Google>, <Company: Google>, <Company: Google>, <Company: Google>, <Company: Company BBB>, <Company: Company CCC>, <Company: Company AAA>]>

In [44]:
Company.objects.order_by(Length("name").desc())

<QuerySet [<Company: Company AAA>, <Company: Company BBB>, <Company: Company CCC>, <Company: Google>, <Company: Google>, <Company: Google>, <Company: Google>]>

In [45]:
Company.objects.order_by("-name", "num_employees")

<QuerySet [<Company: Google>, <Company: Google>, <Company: Google>, <Company: Google>, <Company: Company CCC>, <Company: Company BBB>, <Company: Company AAA>]>

In [51]:
company = Company.objects.annotate(
     need_chairs=GreaterThan(F("num_employees"), F("num_chairs")),
)
for c in company:
    print(f"Name: {c.name} | {c.need_chairs}")

Name: Company AAA | False
Name: Company BBB | True
Name: Company CCC | True
Name: Google | False


In [13]:
from books.models import *

Book.objects.filter(publisher__name="Penguin Books").count()

20

In [14]:
from django.db.models import Avg, Max

Book.objects.aggregate(Avg("price", default=0))

{'price__avg': Decimal('9.7018644067796610')}

In [15]:
Book.objects.aggregate(Max("price", default=0))

{'price__max': Decimal('14.99')}

In [18]:
publisher = Publisher.objects.annotate(avg_price=Avg("book__price"))
for public in publisher.values():
    print(public)

{'id': 2, 'name': 'Oxford University Press', 'avg_price': Decimal('9.7464102564102564')}
{'id': 1, 'name': 'Penguin Books', 'avg_price': Decimal('9.6150000000000000')}


In [22]:
authors = Author.objects.annotate(book_num=Count("book"))
for a in authors.values(): print(a)

{'id': 22, 'name': 'Joseph Heller', 'age': 76, 'book_num': 1}
{'id': 15, 'name': 'Lewis Carroll', 'age': 65, 'book_num': 0}
{'id': 19, 'name': 'Charlotte Brontë', 'age': 38, 'book_num': 0}
{'id': 5, 'name': 'J.D. Salinger', 'age': 91, 'book_num': 5}
{'id': 4, 'name': 'Jane Austen', 'age': 41, 'book_num': 4}
{'id': 10, 'name': 'Charles Dickens', 'age': 58, 'book_num': 4}
{'id': 6, 'name': 'Herman Melville', 'age': 72, 'book_num': 2}
{'id': 14, 'name': 'J.R.R. Tolkien', 'age': 81, 'book_num': 1}
{'id': 13, 'name': 'J.K. Rowling', 'age': 57, 'book_num': 1}
{'id': 2, 'name': 'George Orwell', 'age': 46, 'book_num': 4}
{'id': 7, 'name': 'Leo Tolstoy', 'age': 82, 'book_num': 4}
{'id': 20, 'name': 'Emily Brontë', 'age': 30, 'book_num': 3}
{'id': 1, 'name': 'F. Scott Fitzgerald', 'age': 44, 'book_num': 4}
{'id': 18, 'name': 'Ray Bradbury', 'age': 91, 'book_num': 1}
{'id': 8, 'name': 'Homer', 'age': 50, 'book_num': 3}
{'id': 11, 'name': 'Gabriel García Márquez', 'age': 87, 'book_num': 2}
{'id': 

In [27]:
authors = Author.objects.filter(
    book__rating__gt=4.5
).annotate(
    book_num=Count("book")
)
for i in authors.values(): print(i)

{'id': 4, 'name': 'Jane Austen', 'age': 41, 'book_num': 1}
{'id': 6, 'name': 'Herman Melville', 'age': 72, 'book_num': 1}
{'id': 2, 'name': 'George Orwell', 'age': 46, 'book_num': 2}
{'id': 7, 'name': 'Leo Tolstoy', 'age': 82, 'book_num': 2}
{'id': 20, 'name': 'Emily Brontë', 'age': 30, 'book_num': 1}
{'id': 1, 'name': 'F. Scott Fitzgerald', 'age': 44, 'book_num': 1}
{'id': 9, 'name': 'Fyodor Dostoevsky', 'age': 59, 'book_num': 1}


In [28]:
pubs = Publisher.objects.annotate(num_books=Count("book")).order_by("-num_books")[:5]
pubs[0].num_books

39